In [4]:
from database import Database, LoadDatabase
import numpy as np

In [5]:
DB = LoadDatabase()

In [1]:
import time
class CFRatingPredictor:
    def __init__(self, db):
        self.standings = db.standings
        self.contests = list(db.contests.index)
        self.contests.reverse()
    
    def intDiv(self, x,y):
        return - ((-x) // y) if x < 0 else x // y
    
    def Pij(self, ri, rj):
        return 1.0 / (1.0 + np.power(10.0, (rj-ri)/400.0))
    
    def getSeed(self, rating, contestant, contestStandings):
        prob = lambda r : self.Pij(r, rating)
        vecProb = np.vectorize(prob)
        oldRatings = contestStandings['oldRating'].to_numpy()
        seed = np.sum(vecProb(oldRatings)) - self.Pij(contestant['oldRating'],rating) + 1
        return seed
    
    def getSeedCached(self, npCS):
        cache = {}
        def seedCacher(rating):
            if rating not in cache:
                prob = lambda r: self.Pij(r, rating)
                vecProb = np.vectorize(prob)
                cache[rating] = np.sum(vecProb(npCS)) + 0.5
            return cache[rating]
        
        return seedCacher
    
    def getInitialRatingChange(self, contestant, contestStandings):
        rank = contestant['rank']
        oldRating = contestant['oldRating']
        seed = self.getSeed(oldRating, contestant, contestStandings)
        midRank = np.sqrt(rank*seed)
        R = self.binarySearch(midRank, lambda r: self.getSeed(r, contestant, contestStandings))
        delta = self.intDiv(R-oldRating,2)
        return delta
    
    def binarySearch(self, value, f, left=1, right=8000):
        while right - left > 1:
            mid = self.intDiv(left+right,2)
            print(left, right)
            print(mid, f(mid), value)
            if f(mid) < value:
                right = mid
            else:
                left = mid
        return left
     
    def fightAgainstInflation(self, contestStandings, deltas):
        contestantsSorted = contestStandings.sort_values(by=['oldRating'], ascending=False)
        pplCount = len(contestStandings)
        topPplCount = (min(pplCount, 4*(np.rint(np.sqrt(pplCount))))).astype(int)
        topContestans = contestantsSorted.head(topPplCount)
        sum = np.sum([deltas[i] for i in topContestans.index])
        inc = min(0, max(-10, -(self.intDiv(sum, topPplCount))))
        print(inc)
        return deltas + inc
    
    def fixToSumZero(self, contestStandings, deltas):
        sum = np.sum(deltas)
        inc = -self.intDiv(sum, len(contestStandings)) - 1
        return deltas + inc
    
    def processContest(self, contestStandings):
        deltas = contestStandings.apply(lambda c: self.getInitialRatingChange(c, contestStandings), axis=1)   
        deltas = self.fixToSumZero(contestStandings, deltas)
        deltas = self.fightAgainstInflation(contestStandings, deltas)
        return deltas
    
    def calcErrorContest(self, contestStandings, errCalc):
        npCS = contestStandings['oldRating'].to_numpy()
        seedCalculator = self.getSeedCached(npCS)
        npSeedCalc = np.vectorize(seedCalculator)
        oldRating = contestStandings['oldRating'].to_numpy()
        expectedRanks = npSeedCalc(oldRating)
        actualRanks = contestStandings['rank'].to_numpy()
        vecErrCalc = np.vectorize(errCalc)
        return np.sum(vecErrCalc(expectedRanks, actualRanks)) / len(contestStandings)
    
    def genErrRateDic(self, errCalc):
        errRateDic = {}
        for key in self.contests:
            if key < 1000:
                continue
            contestStandings = self.standings[key]
            print("Contest ", key, " started")
            print("There are ", len(contestStandings), " participants in this contest")
            errRateDic[key] = self.calcErrorContest(contestStandings, errCalc)
            print("Contest ", key, " is done!")
        return errRateDic

def GenCFRatingErrorRates(DB, errCalc):
    rp = CFRatingPredictor(DB)
    return rp.genErrRateDic(errCalc)

def AnadiErrorRate(a, b):
    return abs(a-b)

def LogErrorRate(a, b):
    return np.abs(np.log(a) - np.log(b))

In [6]:
rp = CFRatingPredictor(DB)

In [19]:
errors = rp.genErrRateDic(AnadiErrorRate)

Contest  1004  started
There are  3908  participants in this contest
Contest  1004  is done!
Contest  1007  started
There are  542  participants in this contest
Contest  1007  is done!
Contest  1008  started
There are  4633  participants in this contest
Contest  1008  is done!
Contest  1010  started
There are  587  participants in this contest
Contest  1010  is done!
Contest  1011  started
There are  4536  participants in this contest
Contest  1011  is done!
Contest  1012  started
There are  495  participants in this contest
Contest  1012  is done!
Contest  1013  started
There are  3449  participants in this contest
Contest  1013  is done!
Contest  1019  started
There are  522  participants in this contest
Contest  1019  is done!
Contest  1020  started
There are  4750  participants in this contest
Contest  1020  is done!
Contest  1023  started
There are  5299  participants in this contest
Contest  1023  is done!
Contest  1025  started
There are  5249  participants in this contest
Conte

Contest  1168  is done!
Contest  1169  started
There are  4677  participants in this contest
Contest  1169  is done!
Contest  1172  started
There are  553  participants in this contest
Contest  1172  is done!
Contest  1173  started
There are  5211  participants in this contest
Contest  1173  is done!
Contest  1174  started
There are  5556  participants in this contest
Contest  1174  is done!
Contest  1178  started
There are  5584  participants in this contest
Contest  1178  is done!
Contest  1179  started
There are  510  participants in this contest
Contest  1179  is done!
Contest  1180  started
There are  5063  participants in this contest
Contest  1180  is done!
Contest  1181  started
There are  4669  participants in this contest
Contest  1181  is done!
Contest  1182  started
There are  5543  participants in this contest
Contest  1182  is done!
Contest  1185  started
There are  6123  participants in this contest
Contest  1185  is done!
Contest  1188  started
There are  518  participa

Contest  1326  is done!
Contest  1329  started
There are  1273  participants in this contest
Contest  1329  is done!
Contest  1330  started
There are  10889  participants in this contest
Contest  1330  is done!
Contest  1332  started
There are  12012  participants in this contest
Contest  1332  is done!
Contest  1333  started
There are  12810  participants in this contest
Contest  1333  is done!
Contest  1336  started
There are  1406  participants in this contest
Contest  1336  is done!
Contest  1337  started
There are  14830  participants in this contest
Contest  1337  is done!
Contest  1338  started
There are  1392  participants in this contest
Contest  1338  is done!
Contest  1339  started
There are  12032  participants in this contest
Contest  1339  is done!
Contest  1341  started
There are  15390  participants in this contest
Contest  1341  is done!
Contest  1348  started
There are  15215  participants in this contest
Contest  1348  is done!
Contest  1349  started
There are  1248 

In [20]:
for key, ers in errors.items():
    print(key, ers)

1004 597.0021365793608
1007 119.5191231738017
1008 780.4003378687202
1010 114.57802399345375
1011 738.3224919397705
1012 93.30308518068355
1013 584.5190901055213
1019 101.42006480100594
1020 874.1334594796296
1023 772.3748906402868
1025 817.9503601347543
1028 624.9517240902012
1030 578.6867742815879
1031 509.83444982979665
1033 454.3205171793905
1034 98.01885658738667
1037 669.1885659471751
1038 834.3174846807019
1039 63.436467081044654
1040 835.0953326166469
1041 817.8918728532531
1042 456.6745424476406
1043 587.9307479663289
1044 4.755834197935023
1047 998.9145073122107
1053 88.29739197691664
1054 644.4860889330862
1055 475.40192601440293
1056 381.91717547081004
1058 698.4323169965769
1059 740.6224520629119
1060 524.726909703892
1061 837.4378176446708
1062 599.2161353356456
1063 100.11935450701577
1064 671.2551058124644
1067 78.24586340317525
1068 664.9789491259996
1071 89.65458358301503
1072 493.37058535049437
1074 60.01677232278441
1075 400.8066901409297
1080 591.478608246016
1081 

In [7]:
errors = rp.genErrRateDic(LogErrorRate)

Contest  1004  started
There are  3908  participants in this contest
Contest  1004  is done!
Contest  1007  started
There are  542  participants in this contest
Contest  1007  is done!
Contest  1008  started
There are  4633  participants in this contest
Contest  1008  is done!
Contest  1010  started
There are  587  participants in this contest
Contest  1010  is done!
Contest  1011  started
There are  4536  participants in this contest
Contest  1011  is done!
Contest  1012  started
There are  495  participants in this contest
Contest  1012  is done!
Contest  1013  started
There are  3449  participants in this contest
Contest  1013  is done!
Contest  1019  started
There are  522  participants in this contest
Contest  1019  is done!
Contest  1020  started
There are  4750  participants in this contest
Contest  1020  is done!
Contest  1023  started
There are  5299  participants in this contest
Contest  1023  is done!
Contest  1025  started
There are  5249  participants in this contest
Conte

Contest  1168  is done!
Contest  1169  started
There are  4677  participants in this contest
Contest  1169  is done!
Contest  1172  started
There are  553  participants in this contest
Contest  1172  is done!
Contest  1173  started
There are  5211  participants in this contest
Contest  1173  is done!
Contest  1174  started
There are  5556  participants in this contest
Contest  1174  is done!
Contest  1178  started
There are  5584  participants in this contest
Contest  1178  is done!
Contest  1179  started
There are  510  participants in this contest
Contest  1179  is done!
Contest  1180  started
There are  5063  participants in this contest
Contest  1180  is done!
Contest  1181  started
There are  4669  participants in this contest
Contest  1181  is done!
Contest  1182  started
There are  5543  participants in this contest
Contest  1182  is done!
Contest  1185  started
There are  6123  participants in this contest
Contest  1185  is done!
Contest  1188  started
There are  518  participa

Contest  1326  is done!
Contest  1329  started
There are  1273  participants in this contest
Contest  1329  is done!
Contest  1330  started
There are  10889  participants in this contest
Contest  1330  is done!
Contest  1332  started
There are  12012  participants in this contest
Contest  1332  is done!
Contest  1333  started
There are  12810  participants in this contest
Contest  1333  is done!
Contest  1336  started
There are  1406  participants in this contest
Contest  1336  is done!
Contest  1337  started
There are  14830  participants in this contest
Contest  1337  is done!
Contest  1338  started
There are  1392  participants in this contest
Contest  1338  is done!
Contest  1339  started
There are  12032  participants in this contest
Contest  1339  is done!
Contest  1341  started
There are  15390  participants in this contest
Contest  1341  is done!
Contest  1348  started
There are  15215  participants in this contest
Contest  1348  is done!
Contest  1349  started
There are  1248 

In [11]:
for key, ers in errors.items():
    print(key, ers)

1004 0.44199403534974935
1007 0.6098474573458977
1008 0.5101841408792666
1010 0.5404802845636614
1011 0.4990223701734181
1012 0.5043935334271632
1013 0.501146498664154
1019 0.5241615087164896
1020 0.5377266387481292
1023 0.4071236785796864
1025 0.4379242991122622
1028 0.3919876610456042
1030 0.6333844847741735
1031 0.6782255290569968
1033 0.37306596848780543
1034 0.538363540821909
1037 0.3733380832221117
1038 0.4818729851964774
1039 0.49330044741556306
1040 0.5567927487907118
1041 0.4644177954014263
1042 0.45330697872429077
1043 0.3518518634451428
1044 0.4395524706894257
1047 0.5479340847318263
1053 0.4689904677412131
1054 0.40693269850027736
1055 0.38901504998329794
1056 0.3367455349776507
1058 0.512122861323746
1059 0.4544071868726169
1060 0.407759665660511
1061 0.46924737905621766
1062 0.42948582503001886
1063 0.5698888251478661
1064 0.5071314492125625
1067 0.5486897616814722
1068 0.49406753328360875
1071 0.5205200913153056
1072 0.509937403326698
1074 0.4477801315021715
1075 0.48136

In [16]:
np.sum(np.array(list(errors.values())))/len(errors)

0.46704327556909464

In [8]:
class CFOptimized(CFRatingPredictor):
    def Pij(self, ri, rj):
        return 1.0 / (1.0 + np.power(10.0, (rj-ri)/1100.0))

In [ ]:
rp = CFOptimized(DB)

In [ ]:
errors = rp.genErrRateDic(LogErrorRate)